# Lade Daten per Twitter API
*Es werden Timeline Daten für jeden aktuellen Bundestags Politiker geladen, der auf Twitter aktiv ist. 
Die Daten werden anschließend in MongoDB abgelegt, wobei jeder Tweet einem Dokument entspricht. Die Namen der User wurden von den Fraktionsseiten gecrawled.*

### API Verbindung aufsetzen

In [1]:
# Tweepy statt twitter Paket
# https://www.earthdatascience.org/courses/earth-analytics-python/using-apis-natural-language-processing-twitter/get-and-use-twitter-data-in-python/

import os
import tweepy as tw
import pandas as pd

# Die folgenden Zugangsaten können bei Twitter Developer abgerufen werden, sobald ein Account erstellt wurde
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token_key = os.getenv('access_token_key')
access_token_secret = os.getenv('access_token_secret')

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

### API Verbindung und speichern in Mongo DB
In der Collection Twitter_mdp wird jeder Tweet mit allen Metainformationen als ein Dokument gespeichert. Es werden Tweets zu allen auf Twitter aktiven Politikern einer Fraktion gesammelt. Die entsprechenden Namen wurden von den Bundestagsfraktionsseiten gecrawled.

Anfang Juni 2020 wurde zur Qualitätsicherung des Datensatzes außerdem eine manuelle Kontrolle durchgeführt. Dabei fiel auf, dass der Großteil von Abgeordneten einen Twitter Account besitzen, dieser aber nicht auf der Fraktionsseite verlinkt wurde. Dies ist der Fall bei den folgenden Politikern:

"nicole_ae_bauer", "Bubendorferfdp", "BrittaDassler", "HerbrandMarkus", "michael_g_link","juergen_martens", "NicoleWestig" 

Die Tweets aus den Timelines dieser Politiker wurden entsprechend im Juni in den Datensatz aufgenommen.

Für die folgenden Namen gab es Auffälligkeiten, weshalb sie nicht enthalten sind (Grund angegeben):
* ruppert_stefan: Zum 1. April 2020 wechselte er als Arbeitsdirektor in den Vorstand von B. Braun 

In [2]:
import pymongo

client = pymongo.MongoClient("mongodb://127.0.0.1:27017/")

db = client['Twitter']
# All_Tweets_collection = db['twitter_mdp_ex_date_proj'] # Flache Collection mit weniger Feldern
All_Tweets_collection = db['Twitter_mdp_extend']         # Basis in der zunächst alle Tweets gesammelt werden

Über den Zeitraum Feb-Aug 2020 wurde der Datensatz zu verschiedenen Zeitpunkten aktualisiert. Dabei ergaben sich folgende Probleme:
* die Option "since" in $\texttt{.user_timeline}$ wird ignoriert wenn man gleichtzeitig "count" oder "page" verwendet
* beim laden in die modifizierte Collection Twitter_mp_datetime wurde ein duplicate error geworfen
* durch die Arbeit auf zwei Computern muss die Datenbank bei Aktualisierung synchronisiert werden

In [3]:
import json

with open('Namen_FDP.json') as json_file:
    FDP = json.load(json_file)
    
#FDP = FDP[48:]

for Abgeordneter in FDP:
    i = 1
    user = Abgeordneter
    print(user)
    all_tweets = []
    while i <= 100 :
        tweets_of_page = api.user_timeline(screen_name = user, tweet_mode = "extended", count = 200, page = i, include_rts = True)
        #print(len(tweets_of_page))
        all_tweets = all_tweets + tweets_of_page
        i=i+1
        if len(tweets_of_page) == 0:
            break    # no more tweets from this user
            
    for tweet in all_tweets:
        All_Tweets_collection.insert_one(tweet._json)
    print(len(all_tweets))

aggelidis_fdp
585
RenataAlt_MdB
690
cad59
1698
jensbeeck
173
JBrandenburgFDP
1457
BraFDP
1901
MarcoBuschmann
3230
busen_mdb
119
DjirSarai
854
christianduerr
1659
HartmutEbbing
540
MarcusFaber
3195
Otto_Fricke
3229
DFoest
3229
Lambsdorff
3199
hacker_fdp
3136
PeterHeidtFDP
388
KatrinHelling
1422
torstenherbst
1223
fdp_hessel
393
GeroHocker
918
HoffmannForest
1721
HoubenReinhard
348
ManuelHoeferlin
3214
ullaihnen
97
GydeJ
1893
c_jung77
3238
MdBKlein
450
DanielaKluckert
1591
carina_konrad
1508
KonstantinKuhle
3198
AKulitz
199
koehler_fdp
1440
ulrichlechte
710
c_lindner
3219
OlliLuksic
3219
TillMansmann
59
ChristophFDP
3222
alexmuellerfdp
915
muellerboehm
196
theliberalfrank
663
_MartinNeumann
2631
reinholdmdb
2823
reuther_bernd
468
th_sattelberger
3210
Chr_Sauter
466
WSchinnenburg
1483
f_schaeffler
3224
mseesternpauly
189
franksitta
3231
JudithSkudelny
407
solms
1884
starkwatzinger
1928
MAStrackZi
3192
bstrasser
2591
KatjaSuding
3111
LindaTeuteberg
1587
EUTheurer
2834
StephanThomae
1779
MT

In [ ]:
import json
with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/FDP_twitter_clean.json') as json_file:
    FDP = json.load(json_file)
print(FDP)

#### Liste aller FDP Politiker ohne Daten in Twitter_mdp

Für manche Politiker findet man keine Daten in Twitter_mdp, wenn man mit dem Screen_name aus der Liste FDP_twitter_clean sucht.

Grund: Die Usernamen in der Liste von der Fraktionsseite (_twitter_clean) weichen ab vom tatsächlichen Screen_name

-> Twitter API ist NICHT case sensitiv, MongoDB und Python aber schon!

=> Option mit RegEx zu suchen (_twitter_Namen) ODER einen Tweet laden und exakten user_screen_name speichern (siehe unten - _twitter_realNames)

In [2]:
# import json
# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/FDP_twitter_clean.json') as json_file:
#     FDP = json.load(json_file)

# FDP_Name = FDP
# mdp_not_mongodb = []

# for name, n in zip(FDP, range(0,len(FDP))):
#     count = 0
#     #cursor = All_Tweets_collection.find({"user.screen_name": {'$regex':name,'$options':'i'}})
#     cursor = All_Tweets_collection.find({"user_screen_name": name})
#     for doc in cursor:
#         count=count+1
        
#     if count == 0:
#         #print(count)
#         print(name)
#         #mdp_not_mongodb.append(name)
#         FDP_Name[n] = All_Tweets_collection.find_one({"user_screen_name": {'$regex':name,'$options':'i'}})['user_screen_name']
   
        
# print(mdp_not_mongodb)
# with open("/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/FDP_twitter_Names.json", "a") as write_file:
# #with open("mdp_not_mongodb_FDP.json", "w") as write_file:
#      json.dump(FDP_Name, write_file)

#### Erstellen der Liste mit den tatsächlichen screen_names 

In [1]:
# import json

# with open('/home/lisa/Darmstadt/Master Arbeit/05_Data/Scrapy/Twitter_user/Twitter_user/spiders/FDP_twitter_clean.json') as json_file:
#     FDP= json.load(json_file)
    
# #FDP = FDP[]
# names = []

# for Abgeordneter in FDP:
#     user = Abgeordneter
#     print(user)
#     all_tweets = api.user_timeline(screen_name = user, count = 1, include_rts = True)
#     for tweet in all_tweets:
#         name = tweet.user.screen_name
#         names.append(name)
        
# print(names)

# with open ("FDP_twitter_realNames", "w") as write_file:
#     json.dump(names, write_file)